<div align="center">
    <font color="0F5298" size="7">
        Deep Learning <br>
    </font>
    <font color="2565AE" size="5">
        CE Department <br>
        Spring 2024 - Prof. Soleymani Baghshah <br>
    </font>
    <font color="3C99D" size="5">
        HW2 Practical <br>
    </font>
    <font color="696880" size="5">
        30 Points
    </font>
</div>


In [ ]:
FULLNAME = 'YOUR NAME'
STD_ID = 'YOUR ID'

In this notebook, we aim to perform **classification** on images from the **CIFAR10** dataset using CNN networks. First, we load the dataset and apply the necessary transformations for normalization and augmentation. After that, we visualize some samples. Once we familiarize ourselves with the dataset, we proceed to design the desired convolutional network, which is explained in the relevant section. After designing the model, we move on to training and evaluating it. At the end of the first section, we analyze the feature space from different perspectives. First, using the KNN method, we examine the closest samples to each other in the feature space. Then, we cluster the data and finally visualize the outputs of the intermediate layers of the model.

In the second part of the notebook, we perform a simple transfer learning task on the trained model from the first section but using a new dataset, **CIFAR100**. To do this, we modify the final layer of the network and retrain it. Further details are provided in the relevant section. Finally, we evaluate the model’s accuracy on the new task and analyze the extracted features and how well the model generalizes. After designing and training the model, we will further analyze the extracted feature space. Finally, we will evaluate the generalization ability of the model and its extracted features on a new dataset,

# CIFAR10 Classification

## Import Libraries

Import needed libraries

In [ ]:
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
import tqdm
from time import time
import random
from sklearn.manifold import TSNE
import numpy as np
from random import sample
import math
import torch.optim as optim
import seaborn as sns

## Device

Set device to work with (GPU or CPU)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## Transforms & Dataset & Dataloader

Here, you should download and load the dataset with the desire transforms. After that, you should split train dataset to train and validation sets. Finally, define the dataloaders for `train`, `validation` and `test`

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:

# Data Transforms
cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2470, 0.2435, 0.2616)

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std),
])

# Load Train Data
full_train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)

# Split Train and Validation Data
val_size = 5000
train_size = len(full_train_dataset) - val_size
train_dataset, val_dataset = torch.utils.data.random_split(full_train_dataset, [train_size, val_size])

# Load Test Data
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# Define Data Loaders
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)


## Visualization

Visualize 5 random images from each class in different columns

- **Hint**:  You can use `plt.subplots` for visualization

In [ ]:

# Find 5 Images from Each Class
fig, axs = plt.subplots(5, len(classes), figsize=(15, 7))
class_counts = {cls: 0 for cls in classes}
for img, label in full_train_dataset:
    cls_name = classes[label]
    if class_counts[cls_name] < 5:
        ax = axs[class_counts[cls_name]][label]
        img_to_show = img.clone() * torch.tensor(cifar10_std).view(3,1,1) + torch.tensor(cifar10_mean).view(3,1,1)
        ax.imshow(img_to_show.permute(1,2,0))
        ax.axis('off')
        if class_counts[cls_name] == 0:
            ax.set_title(cls_name)
        class_counts[cls_name] += 1
    if all(v >= 5 for v in class_counts.values()):
        break
plt.tight_layout()


## Model

Define your model here from scratch (You are not allowed to use the existing models in pytorch)

**NOTICE:** The model that you will have defined outputs a vector containing 10 numbers (for each class). Define a "feature space" that is a vector of size *N* (where *N > 10*) right before the last layer (You can then have a last layer like `nn.Linear(N, 10)`). See the image below to get a better understanding. We will use this later (we want to access the feature space of a sample when the sample is given to the model). The model tries to learn a representation of the samples in this feature space and we will see how good it could do this in later sections.

![Feature Space In Neural Network](https://i.postimg.cc/28Qjcn9D/feature-space-vis.png)

- **Hint I**: Our goal is to get accuracy above *90%* on testset. Our suggestion is to implement ResNet (ResNet18 could be a viable choice)
  - You can learn the network's structure and implementation online (Youtube, ...) and then implmenet it yourself and make changes to enhance it's performance on our task **(YOU SHOULD NOT COPY THE CODE!!! OTHERWISE, YOU'LL BE PENALIZED!!!)**

- **Hint II**: When defining your model, pay attension to the **NOTICE** part in the above. It's also better to read the "Exploring the feature space" section beforehand.  

In [ ]:

# Implement Model
class SimpleResNet(nn.Module):
    def __init__(self, num_classes=10, feature_dim=256):
        super().__init__()
        self.feature_dim = feature_dim
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1,1)),
        )
        self.fc_feat = nn.Linear(256, feature_dim)
        self.fc_out = nn.Linear(feature_dim, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0), -1)
        features = self.fc_feat(x)
        features = F.relu(features)
        logits = self.fc_out(features)
        return logits, features


## Train

### Model instantiation

Create an instance of your model and move it to `device`

In [ ]:

# Define Model
model = SimpleResNet(num_classes=10, feature_dim=256).to(device)


### Criterion & Optimizater

Define `criterion` and `optimizer` (Or `scheduler`)

In [ ]:

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)


### Train loop

Train your model

Tasks:
- [ ] Things that are needed to be printed in each epoch:
  - Number of epoch
  - Train loss
  - Train accuracy
  - Validation loss
  - Validation accuracy
- [ ] save train/validation loss and accuracy (of each epoch) in an array for later usage

In [ ]:

# Implement Training Loop
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs, _ = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss/total, correct/total


In [ ]:

# Implement Validation Loop
def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs, _ = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss/total, correct/total


In [ ]:

# Train The Model
epochs = 10
train_losses, val_losses = [], []
train_accs, val_accs = [], []
for epoch in range(epochs):
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    scheduler.step()
    train_losses.append(tr_loss); val_losses.append(val_loss)
    train_accs.append(tr_acc); val_accs.append(val_acc)
    print(f"Epoch {epoch+1}/{epochs} | Train Loss {tr_loss:.4f} Acc {tr_acc:.3f} | Val Loss {val_loss:.4f} Acc {val_acc:.3f}")


### Save Model

Since changes need to be made to the model later on, it is advisable to save your model to avoid having to retrain it in case of any issues.

In [ ]:

# Save Model
ckpt_path = 'cifar10_model.pth'
torch.save({'model_state': model.state_dict(), 'feature_dim': model.feature_dim}, ckpt_path)
print(f"Model saved to {ckpt_path}")


### Visualize Loss and Accuracy plot

Using the arrays that you have (from task 2 in the above section), visualize two plots: Accuracy plot (train and validation together) and Loss plot (train and validation together)

In [ ]:

# Plot Loss
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Val')
plt.title('Loss')
plt.legend()
plt.grid(True)

# Plot Accuracy
plt.subplot(1,2,2)
plt.plot(train_accs, label='Train')
plt.plot(val_accs, label='Val')
plt.title('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


## Evaluation

Test your trained model (using the Test Dataloader that you have). Our goal is to reach an accuracy above `90%`

In [ ]:

# Run Model on Testset
model.eval()
correct, total = 0, 0
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs, _ = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        all_preds.extend(predicted.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.4f}")


## Visualize incorrectly predicted samples from testset

Visualize *24* random images from testset that are incorrectly predicted by the model. Note that if you used normalization in the transform function for loading the data, you will need to unnormalize the images before displaying them.

In [ ]:

# Plot Samples with Wrong Predicted Classes
wrong_indices = [i for i, (p, l) in enumerate(zip(all_preds, all_labels)) if p != l]
fig, axs = plt.subplots(4, 6, figsize=(12,8))
axs = axs.flatten()
shown = 0
for batch_idx, (images, labels) in enumerate(test_loader):
    for i in range(images.size(0)):
        global_idx = batch_idx * test_loader.batch_size + i
        if global_idx in wrong_indices and shown < len(axs):
            img = images[i].cpu() * torch.tensor(cifar10_std).view(3,1,1) + torch.tensor(cifar10_mean).view(3,1,1)
            axs[shown].imshow(img.permute(1,2,0))
            axs[shown].set_title(f"T:{classes[labels[i]]}
P:{classes[all_preds[global_idx]]}")
            axs[shown].axis('off')
            shown += 1
    if shown >= len(axs):
        break
plt.tight_layout()
plt.show()


## Exploring the feature space

### Calculate the feature space for all training samples

You have trained and evaluated your model. Now, for each sample in the trainset, calculate it's "feature space" discussed in the model section. The result of this section should be a tensor of size `(50000, N)` saved in a variable (for later usage)

- **Hint 1:** define a tensor with dimension `(50000, N)` where *50000* is the size of the trainset and *N* is the dimension of the feature space

- **Hint 2:** Pay attension to the `shuffle` attribute of your train dataloader (If needed)

In [ ]:

# Find Features and Put Them in One Dimensional List
model.eval()
train_features = []
train_labels = []
with torch.no_grad():
    for images, labels in torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=False, num_workers=2):
        images = images.to(device)
        _, feats = model(images)
        train_features.append(feats.cpu())
        train_labels.append(labels)
train_features = torch.cat(train_features, dim=0)
train_labels = torch.cat(train_labels, dim=0)
print(train_features.shape)


### K Nearest Neighbor in feature space

You already have calculated the feature spaces for trainset ($S$) in the previous section

1. Get 5 random samples from testset which are correctly predicted by the model.
2. for each sample, calculate it's "feature space" ($X$)
3. for each sample, calculate it's *5* nearest neighbors in "feature space" in the trainset (by comparing $X$ to each row in $S$) and visualize them

**Note:** Your visualization should be something like the below picture

**Hint:** To find the nearest neighbors in the feature space, you can use any library of your choice.

In [ ]:

# Find Features List for Test Samples
from sklearn.neighbors import NearestNeighbors
model.eval()
correct_samples = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        logits, feats = model(images)
        preds = logits.argmax(dim=1).cpu()
        for img, lab, pred, feat in zip(images.cpu(), labels, preds, feats.cpu()):
            if lab == pred:
                correct_samples.append((img, lab.item(), feat))
        if len(correct_samples) >= 5:
            break

feats_mat = torch.stack([f for _,_,f in correct_samples])
labels_correct = [l for _, l, _ in correct_samples]

knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(train_features)
dists, idxs = knn.kneighbors(feats_mat)

fig, axs = plt.subplots(len(correct_samples), 6, figsize=(12,8))
for row, (img, lab, feat) in enumerate(correct_samples):
    img_disp = img.cpu() * torch.tensor(cifar10_std).view(3,1,1) + torch.tensor(cifar10_mean).view(3,1,1)
    axs[row,0].imshow(img_disp.permute(1,2,0))
    axs[row,0].set_title(f"Query:{classes[lab]}")
    axs[row,0].axis('off')
    for k in range(5):
        tr_idx = idxs[row, k]
        tr_img, tr_lab = full_train_dataset[tr_idx]
        tr_img_disp = tr_img * torch.tensor(cifar10_std).view(3,1,1) + torch.tensor(cifar10_mean).view(3,1,1)
        axs[row,k+1].imshow(tr_img_disp.permute(1,2,0))
        axs[row,k+1].set_title(classes[tr_lab])
        axs[row,k+1].axis('off')
plt.tight_layout()
plt.show()


### TSNE

1. Sample $M$ ($2000$ would be enought) random samples from the trainset feature space (calculated in the above sections)
2. Now you a vector of size `(M, N)` where $N$ is the dimension of the feature space
3. Using TSNE reduce $N$ to $2$ (Now you have a vector of size `(M, 2)`)
4. Print the shape of the output

**Hint:** You can use `sklearn.manifold.TSNE`

In [ ]:

# Get Samples
M = 2000
indices = torch.randperm(train_features.size(0))[:M]
feat_sample = train_features[indices]
label_sample = train_labels[indices]

# Use TSNE
tsne = TSNE(n_components=2, learning_rate='auto', init='pca', perplexity=30)
feat_tsne = tsne.fit_transform(feat_sample.numpy())
print(feat_tsne.shape)


Visualize the points in a 2D plane (Set color of each point based on it's class)

**Notice:** Your visualization should be something like the below image

**Hint:** Use `plt.scatter(x, y, c=labels)`

In [ ]:

# Plot Results
plt.figure(figsize=(8,6))
scatter = plt.scatter(feat_tsne[:,0], feat_tsne[:,1], c=label_sample.numpy(), cmap='tab10', s=10)
plt.legend(*scatter.legend_elements(), title="Classes", bbox_to_anchor=(1.05,1), loc='upper left')
plt.title('t-SNE of Feature Space')
plt.tight_layout()
plt.show()


### Feature Map


In this part, we are going to visualize the output of one of the convolutional layers to see what features they focus on.

First, let's select a random image from dataset.

In [ ]:

# Select an Image
sample_img, sample_label = test_dataset[random.randrange(len(test_dataset))]
plt.imshow((sample_img * torch.tensor(cifar10_std).view(3,1,1) + torch.tensor(cifar10_mean).view(3,1,1)).permute(1,2,0))
plt.title(f"Label: {classes[sample_label]}")
plt.axis('off')
plt.show()


Now, we are going to *clip* our model at different points to get different intermediate representation.
* Clip your model at least at one point and plot the filters output. You can use the output of first Resnet block.

In order to clip the model, you can use `model.children()` method. For example, to get output only after the first 2 layers, you can do:

```
clipped = nn.Sequential(
    *list(model.children()[:2])
)
intermediate_output = clipped(input)
```



In [ ]:

# Get Intermediate Output
clipped = nn.Sequential(*list(model.children())[:3])  # up to conv3
with torch.no_grad():
    inter_out = clipped(sample_img.unsqueeze(0).to(device)).cpu()


In [ ]:

# Plot Intermediate Output
plot_intermediate_output(inter_out, title='Feature maps after clipping')


## CIFAR100

In this section, we aim to test the trained model on a different dataset. For this purpose, we will use the CIFAR100 dataset, which is similar to CIFAR10 but has different types and numbers of classes. In order for the model to perform well on the new dataset, we need to modify the last layer of the model. As you know from the previous section, the last layer of the model is a linear layer that maps the features to the number of classes. In this section, due to the increase in the number of classes, we plan to modify this layer and train the new linear layer with the new dataset. Note that all other layers and weights of the model will remain fixed and unchanged; only the last layer will be retrained.

### Dataset & Dataloader

Here, you should download and load the dataset with the desire transforms.

In [ ]:

# Data Transforms
cifar100_mean = (0.5071, 0.4867, 0.4408)
cifar100_std = (0.2675, 0.2565, 0.2761)

train_transform_100 = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(cifar100_mean, cifar100_std),
])

test_transform_100 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar100_mean, cifar100_std),
])

# Load Train and Test Data
cifar100_train = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform_100)
cifar100_test = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform_100)

# Define Data Loaders
train_loader_100 = torch.utils.data.DataLoader(cifar100_train, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_loader_100 = torch.utils.data.DataLoader(cifar100_test, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)


In [ ]:
classes = [
    "apple", "aquarium_fish", "baby", "bear", "beaver", "bed", "bee",
    "beetle", "bicycle", "bottle", "bowl", "boy", "bridge", "bus", "butterfly", "camel", "can", "castle", "caterpillar", "cattle", "chair", "chimpanzee",
    "clock", "cloud", "cockroach", "couch", "crab", "crocodile", "cup", "dinosaur", "dolphin", "elephant", "flatfish", "forest", "fox", "girl", "hamster",
    "house", "kangaroo", "keyboard", "lamp", "lawn_mower", "leopard", "lion", "lizard", "lobster", "man", "maple_tree", "motorcycle", "mountain", "mouse",
    "mushroom", "oak_tree", "orange", "orchid", "otter", "palm_tree", "pear", "pickup_truck", "pine_tree", "plain", "plate", "poppy", "porcupine", "possum",
    "rabbit", "raccoon", "ray", "road", "rocket", "rose", "sea", "seal", "shark", "shrew", "skunk", "skyscraper", "snail", "snake", "spider",
    "squirrel", "streetcar", "sunflower", "sweet_pepper", "table", "tank", "telephone", "television", "tiger", "tractor", "train",
    "trout", "tulip", "turtle", "wardrobe", "whale", "willow_tree", "wolf", "woman", "worm"
]
print(len(classes))

### Visualization

Visualize 1 random images from each class.

- **Hint**:  You can use `plt.subplots` for visualization

In [ ]:

# Find One Image from Each Class
fig, axs = plt.subplots(10, 10, figsize=(18,18))
seen = {}
for img, lbl in cifar100_train:
    if lbl not in seen:
        seen[lbl] = img
    if len(seen) == len(classes):
        break
for idx, (lbl, img) in enumerate(seen.items()):
    r, c = divmod(idx, 10)
    img_disp = img * torch.tensor(cifar100_std).view(3,1,1) + torch.tensor(cifar100_mean).view(3,1,1)
    axs[r][c].imshow(img_disp.permute(1,2,0))
    axs[r][c].set_title(classes[lbl], fontsize=7)
    axs[r][c].axis('off')
plt.tight_layout()


### Modify Model

Change the final linear layer of the model according to the new number of classes And freeze all other layers.
- Do not forgot to move model to `device`

In [ ]:
# Load Pretrained Model

# Freeze All Layers

# Modify The Last Linear Layer

# Move Model to Device


### Criterion & Optimizater

Define `criterion` and `optimizer` (Or `scheduler`)

In [ ]:

# Define Loss and Optimizer
criterion_100 = nn.CrossEntropyLoss()
optimizer_100 = optim.Adam(model.fc_out.parameters(), lr=1e-3)


### Train

Train the Model (Only Last Layer)

In [ ]:

# Train The Model
num_epochs_100 = 5
model.train()
for param in model.parameters():
    param.requires_grad = False
for param in model.fc_out.parameters():
    param.requires_grad = True

for epoch in range(num_epochs_100):
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader_100:
        images, labels = images.to(device), labels.to(device)
        optimizer_100.zero_grad()
        outputs, _ = model(images)
        loss = criterion_100(outputs, labels)
        loss.backward()
        optimizer_100.step()
        running_loss += loss.item() * labels.size(0)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()
    print(f"Epoch {epoch+1}/{num_epochs_100} Loss {running_loss/total:.4f} Acc {correct/total:.3f}")


### Test

Evaluate the Model on CIFAR-100 Test Set. 40% accuracy is sufficient.


In [ ]:

# Evaluate Model on CIFAR100
model.eval()
correct, total = 0, 0
all_preds_100, all_labels_100 = [], []
with torch.no_grad():
    for images, labels in test_loader_100:
        images, labels = images.to(device), labels.to(device)
        outputs, _ = model(images)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()
        all_preds_100.extend(preds.cpu().tolist())
        all_labels_100.extend(labels.cpu().tolist())

acc_100 = correct/total
print(f"CIFAR100 Test Accuracy: {acc_100:.3f}")


### Question
You might think that 40% accuracy is quite low. However, first of all, consider that the classification is done over 100 classes. The accuracy of a random model in this case is 1%. Also, we only changed one linear layer of the model, and the rest of the weights remained unchanged. What do you think is the reason the model can achieve a reasonably good generalization ability on a completely new dataset with just the change of one linear layer at the end?

Answer:

### Visualize incorrectly predicted samples from testset

Visualize *10* random images from testset that are incorrectly predicted by the model

In [ ]:

# Plot Samples with Wrong Predicted Classes
wrong_indices_100 = [i for i, (p,l) in enumerate(zip(all_preds_100, all_labels_100)) if p != l]
fig, axs = plt.subplots(2,5, figsize=(12,5))
axs = axs.flatten()
shown=0
for batch_idx, (images, labels) in enumerate(test_loader_100):
    for i in range(images.size(0)):
        gidx = batch_idx*test_loader_100.batch_size + i
        if gidx in wrong_indices_100 and shown < len(axs):
            img = images[i].cpu() * torch.tensor(cifar100_std).view(3,1,1) + torch.tensor(cifar100_mean).view(3,1,1)
            axs[shown].imshow(img.permute(1,2,0))
            axs[shown].set_title(f"T:{classes[labels[i]]}
P:{classes[all_preds_100[gidx]]}", fontsize=8)
            axs[shown].axis('off')
            shown += 1
    if shown >= len(axs):
        break
plt.tight_layout()
plt.show()


### Plot accuracy for each class

Plot accuracy of model on testset for each class.

In [ ]:

# Calculate Accuracy for Each Class
class_correct = [0 for _ in range(len(classes))]
class_total = [0 for _ in range(len(classes))]
for pred, lbl in zip(all_preds_100, all_labels_100):
    class_total[lbl] += 1
    if pred == lbl:
        class_correct[lbl] += 1
class_acc = [c/t if t>0 else 0 for c, t in zip(class_correct, class_total)]

# Plot Class-Wise Accuracy
plt.figure(figsize=(12,5))
plt.bar(range(len(classes)), class_acc)
plt.xticks(rotation=90, ticks=range(len(classes)), labels=classes, fontsize=6)
plt.ylabel('Accuracy')
plt.title('Per-class accuracy on CIFAR100')
plt.tight_layout()
plt.show()


### The classes with the best and worst accuracy

Based on the results from the previous section, obtain the 5 classes with the best accuracy and the 5 classes with the worst accuracy on the testset, and display one sample from each of them.

In [ ]:

# Find Top 5 Best and Worst Performing Classes
acc_tensor = torch.tensor(class_acc)
best_idx = torch.topk(acc_tensor, 5).indices.tolist()
worst_idx = torch.topk(acc_tensor, 5, largest=False).indices.tolist()
print("Best:", [classes[i] for i in best_idx])
print("Worst:", [classes[i] for i in worst_idx])

# Plot a Sample Image From Each of The Best and Worst Performing Classes
fig, axs = plt.subplots(2,5, figsize=(12,6))
for j, idx in enumerate(best_idx):
    img, _ = next(x for x in cifar100_test if x[1]==idx)
    img_disp = img * torch.tensor(cifar100_std).view(3,1,1) + torch.tensor(cifar100_mean).view(3,1,1)
    axs[0,j].imshow(img_disp.permute(1,2,0))
    axs[0,j].set_title(f"Best: {classes[idx]}")
    axs[0,j].axis('off')
for j, idx in enumerate(worst_idx):
    img, _ = next(x for x in cifar100_test if x[1]==idx)
    img_disp = img * torch.tensor(cifar100_std).view(3,1,1) + torch.tensor(cifar100_mean).view(3,1,1)
    axs[1,j].imshow(img_disp.permute(1,2,0))
    axs[1,j].set_title(f"Worst: {classes[idx]}")
    axs[1,j].axis('off')
plt.tight_layout()
plt.show()


### Question
What do you think is the reason for the significant accuracy difference between different classes? What differences do you observe between the classes with the best and worst accuracy? Can you provide an analysis of the results and relate them to the model’s feature space?

Answer: